### Скрипт для решения задачи автоматизации отчетности

In [2]:
import pandas as pd
import sqlite3

CON = sqlite3.connect('example.db')

In [3]:
# Выгрузка данных с сайта
def extract_data(url):
    """ Extract CSV
    """
    return pd.read_csv(url)

# Группировка данных
def transform_data(data, group, agreg):
    """ Group by data
    """
    return data.groupby(group).agg(agreg).reset_index()

# Загрузка в базу данных
def load_data(data, table_name, conn=CON):
    """ Load to DB
    """
    data["insert_time"] = pd.to_datetime("now")
    data.to_sql(table_name, conn, if_exists='replace', index=False)

# Отправка данных на почту
def send_email(data, cred, host, port, to, From):
    """ Send to email
    """ 
    _send_email(data=data, cred=cred, host=host, to=to, From=From, port=port)

In [4]:
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import smtplib

HOST = "smtp.yandex.ru"
TO = "ejiksam-1@yandex.ru"
FROM = "ejiksam-1@yandex.ru"


def html_pretty(df):
    """ Pretty html dataframe
    """
    return """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(df.to_html())

def _send_email(data, cred, host, port, to, From):
    """ Send DF to email
    """

    msg = MIMEMultipart()
    part = MIMEText(html_pretty(data), 'html')
    msg.attach(part)

    server = smtplib.SMTP(host, port)
    server.starttls()
    server.login(cred[0], cred[1])
    server.sendmail(From, to, msg.as_string())
    server.quit()

In [13]:
# Extract
data = extract_data("https://raw.githubusercontent.com/dm-novikov/stepik_airflow_course/main/data/data.csv")

In [14]:
data

,A,B,C,D
0,spam,alpha,2013-01-02,-0.813103
1,eggs,beta,2013-01-02,1.091019
2,spam,gamma,2013-01-01,-0.403081
3,eggs,alpha,2013-01-02,0.949538
4,spam,beta,2013-01-02,-0.060987
5,eggs,gamma,2013-01-03,0.388832
6,spam,alpha,2013-01-02,1.411927
7,eggs,beta,2013-01-02,-2.709758
8,spam,gamma,2013-01-02,-0.652190
9,eggs,alpha,2013-01-03,0.916228


In [15]:
# Transform
data = transform_data(data, group=['A', 'B', 'C'], agreg={"D": sum})

In [16]:
data

,A,B,C,D
0,eggs,alpha,2013-01-01,0.027503
1,eggs,alpha,2013-01-02,0.949538
2,eggs,alpha,2013-01-03,0.916228
3,eggs,beta,2013-01-02,-2.597512
4,eggs,beta,2013-01-03,2.004210
5,eggs,gamma,2013-01-01,1.747132
6,eggs,gamma,2013-01-03,1.940870
7,spam,alpha,2013-01-01,-1.482974
8,spam,alpha,2013-01-02,-1.714667
9,spam,beta,2013-01-01,-0.965471


In [8]:
# Load to DB
load_data(data, "table")

C:\Users\Ejik\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:2224: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


In [10]:
# Send Email
send_email(data, cred=("ejiksam-1@yandex.ru", "Vino123"), host=HOST, port=587, to=TO, From=FROM)